In [ ]:
# SPDX-FileCopyrightText: 2024 Universidad Autónoma de Madrid, Madrid (Spain)
# SPDX-License-Identifier: EUPL-1.2

"""
This script automates the calibration of hyperspectral images.
It allows processing multiple hyperspectral image directories at once.

Written by: Claudia Fournier
Revised date: 24.10.2024
"""

# Import libraries

In [2]:
import os
import numpy as np
import spectral
from scipy.ndimage import zoom
from spectral import envi

# Define the funcitons

In [19]:
# 1) Loading the raw image and references

def load_hyperspectral_image(path):
    """Load a hyperspectral image using the spectral library."""
    return spectral.open_image(path).load()

In [20]:
# 2) Resizing the references
def resize_hyperspectral_image(image, target_shape):
    """
    Resize a hyperspectral image to match the target shape.
    
    Args:
        image (numpy.ndarray): The hyperspectral image to resize.
        target_shape (tuple): The target shape (rows, columns, wavelengths).
    
    Returns:
        numpy.ndarray: Resized image.
    """
    factors = (
        target_shape[0] / image.shape[0],
        target_shape[1] / image.shape[1],
        1
    )
    return zoom(image, factors, order=1)  # Use bilinear interpolation

In [22]:
# 3) Applying calibraiton equation
def apply_calibration(raw_image, white_ref, dark_ref):
    """
    Apply the calibration equation to the raw hyperspectral image.
    
    Args:
        raw_image (numpy.ndarray): The raw hyperspectral image.
        white_ref (numpy.ndarray): The white reference image.
        dark_ref (numpy.ndarray): The dark reference image.
    
    Returns:
        numpy.ndarray: The calibrated hyperspectral image with values clipped between 0 and 1.
    """
    calibrated_image = (raw_image - dark_ref) / (white_ref - dark_ref)
    # Clip values to valid range [0, 1]
    calibrated_image = np.clip(calibrated_image, 0, 1)
    return calibrated_image

In [45]:
# 4) Saving the calibrated image with per-image folder creation
def save_hyperspectral_image(image_array, root_directory, image_name, metadata=None):
    """
    Save a hyperspectral image using the spectral library in a dedicated folder inside the 'calibrated' directory.
    
    Args:
        image_array (numpy.ndarray): Calibrated hyperspectral image data.
        root_directory (str): Directory where the image should be saved.
        image_name (str): Name of the image file (without extension).
        metadata (dict): Metadata to attach to the saved image.
    """
    # Create the "calibrated" directory if it doesn't exist
    calibrated_dir = os.path.join(root_directory, "calibrated")
    
    # Create a subdirectory inside "calibrated" for each image
    image_output_dir = os.path.join(calibrated_dir, image_name)
    os.makedirs(image_output_dir, exist_ok=True)
    
    # Define the full path to save the image inside its specific subdirectory
    file_path = os.path.join(image_output_dir, f"{image_name}_calibrated.hdr")
    
    # Save the image using the spectral library, and use the 'force' keyword to overwrite if the file exists
    try:
        envi.save_image(file_path, image_array, dtype=np.float32, metadata=metadata, force=True)
    except Exception as e:
        print(f"Error saving hyperspectral image to {file_path}: {e}")

In [46]:
# 5) Processing a directory    
def process_directory(directory):
    """
    Process all hyperspectral image folders within the given directory.
    
    Args:
        directory (str): Root directory containing hyperspectral images and references.
    """
    for root, _, files in os.walk(directory):
        # Filter for hyperspectral image files
        raw_image_file = next((f for f in files if f.endswith(".hdr") and "WHITEREF" not in f and "DARKREF" not in f), None)
        white_ref_file = next((f for f in files if "WHITEREF" in f), None)
        dark_ref_file = next((f for f in files if "DARKREF" in f), None)
        
        if raw_image_file and white_ref_file and dark_ref_file:
            # Define paths
            raw_image_path = os.path.join(root, raw_image_file)
            white_ref_path = os.path.join(root, white_ref_file)
            dark_ref_path = os.path.join(root, dark_ref_file)
            
            print(f"Processing {raw_image_file}...")
            
            # Load images
            raw_image = load_hyperspectral_image(raw_image_path)
            white_ref = load_hyperspectral_image(white_ref_path)
            dark_ref = load_hyperspectral_image(dark_ref_path)
            
            # Resize references if necessary
            if white_ref.shape != raw_image.shape:
                white_ref = resize_hyperspectral_image(white_ref, raw_image.shape)
            if dark_ref.shape != raw_image.shape:
                dark_ref = resize_hyperspectral_image(dark_ref, raw_image.shape)
            
            # Apply calibration
            calibrated_image = apply_calibration(raw_image, white_ref, dark_ref)
            
            # Extract the image name (without extension)
            image_name = os.path.splitext(raw_image_file)[0]
            
            # Save calibrated image in the root directory's "calibrated" folder
            # If the calibrated image already exists, it will be overwritten
            save_hyperspectral_image(calibrated_image, directory, image_name, metadata=raw_image.metadata)
            
            print(f"{image_name} has been successfully calibrated and saved.\n")

# Calibrating images in a given directory

In [47]:
# Prompt to insert the directory path or use the default project path
user_input = input("Insert your directory path for hyperspectral images or press Enter to use the default project path: ")

# If the user does not provide input, use the default relative path
if user_input.strip() == "":
    print("No directory provided, using the default project path.")
    # Set the path relative to the project root
    base_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
    # Default directory containing folders with hyperspectral images
    base_directory = os.path.join(base_path, "data")
else:
    # Use the provided directory path
    base_directory = user_input.strip()

# Process all subdirectories
process_directory(base_directory)

Insert your directory path for hyperspectral images or press Enter to use the default project path: 
No directory provided, using the default project path.
Processing 003.hdr...
003 has been successfully calibrated and saved.

